In [1]:
import plotly.graph_objects as go
import ipywidgets as widgets
import networkx as nx
import numpy as np

import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(os.path.dirname(os.path.dirname(current_dir)))
sys.path.append(parent_dir)

import src.data.Gmap
sys.path.append(parent_dir + '/test')
from gmap import Test

sys.path.insert(0, parent_dir) 

T, gp = Test.sheaf_nodes()

nodescorres = {}
nodes = []
xnodes = []
ynodes = []
znodes = []
nbnodes = 0
for n in gp.OC.iter_icells(0):
    nbnodes += 1
    nodescorres[n] = len(nodes)
    nodes.append(n)
    xnodes.append(gp.ET[n][0])
    ynodes.append(gp.ET[n][1])
    znodes.append(gp.ET[n][2])

xedges = []
yedges = []
zedges = []

itri = []
jtri = []
ktri = []
nbedges = 0
for e in gp.OC.iter_icells(1):
    nbedges+=1
    ep0 = gp.OC.get_icell(0, e)
    xedges.append(gp.ET[ep0][0])
    yedges.append(gp.ET[ep0][1])
    zedges.append(gp.ET[ep0][2])
    ep1 = gp.OC.get_icell(0, gp.OC.alpha(0, e))
    xedges.append(gp.ET[ep1][0])
    yedges.append(gp.ET[ep1][1])
    zedges.append(gp.ET[ep1][2])
    xedges.append(None)
    yedges.append(None)
    zedges.append(None)

nbtri = 0
for t in gp.OC.iter_icells(2):
    nbtri += 1
    # print(t)
    cont = True
    t0i = t
    nodes = [nodescorres[gp.OC.get_icell(0, t0i)]]
    while cont:
        t0ip = gp.OC.alpha(1, t0i)
        t0i = gp.OC.alpha(0, t0ip)
        if t == t0i:
            cont = False
        else:
            nodes.append(nodescorres[gp.OC.get_icell(0, t0i)])
    assert len(nodes) == 3
    # print(nodes)
    itri.append(nodes[0])
    jtri.append(nodes[1])
    ktri.append(nodes[2])

# print(len(xnodes))
# print(len(xedges))
# print(len(itri))
node_trace = go.Scatter3d(
    x=xnodes, y=ynodes, z=znodes,
    mode='markers',
    name='nodes',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        colorscale = 'YlGnBu',
        reversescale=True,
        color = [],
        size = 2,
        line_width=2
    )
)

edge_trace = go.Scatter3d(
    x=xedges, y=yedges, z=zedges,
    line= dict(width=5, color='#888'),
    hoverinfo='text',
    name='edges',
    mode='lines')

fig = go.FigureWidget(data=[node_trace, edge_trace,
    go.Mesh3d(
        x=xnodes,
        y=ynodes,
        z=znodes,
        colorbar_title='faces',
        colorscale=[[0, 'gold'],
                    [0.5, 'mediumturquoise'],
                    [1, 'magenta']],
        # Intensity of each vertex, which will be interpolated and color-coded
        # intensity=[0, 0.33, 0.66, 1],
        #intensity = np.linspace(0, 1, len(itri), endpoint=True),
        intensity = np.random.random(len(itri)),
        intensitymode='cell',
        # i, j and k give the vertices of triangles
        # here we represent the 4 triangles of the tetrahedron surface
        i=itri,
        j=jtri,
        k=ktri,
        name='faces',
        showscale=False,
        showlegend=True
    )],
             layout=go.Layout(
                title = '<br>Subdivised gmap ' + str(nbnodes) + " nodes, " + str(nbedges) + " edges and " + str(nbtri) + " faces",
                titlefont_size=16,
                 legend=dict(
                    x=0,
                    y=0.5
                ),
                showlegend=True,
                hovermode='closest',
                margin=dict(b=0,l=0,r=0,t=0),
                #annotations=[ dict(
                #    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                #    showarrow=False,
                #    xref="paper", yref="paper",
                #    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

# the button callback function
def on_button_click(b):
    global gp
    gp = T.extend(gp)
    nodescorres = {}
    nodes = []
    xnodes = []
    ynodes = []
    znodes = []
    nbnodes = 0
    for n in gp.OC.iter_icells(0):
        nbnodes += 1
        nodescorres[n] = len(nodes)
        nodes.append(n)
        xnodes.append(gp.ET[n][0])
        ynodes.append(gp.ET[n][1])
        znodes.append(gp.ET[n][2])

    # print(xnodes)
    # print(ynodes)
    # print(znodes)

    xedges = []
    yedges = []
    zedges = []

    itri = []
    jtri = []
    ktri = []
    nbedges = 0
    for e in gp.OC.iter_icells(1):
        nbedges += 1
        # print("edge")
        # print(e)
        ep0 = gp.OC.get_icell(0, e)
        xedges.append(gp.ET[ep0][0])
        yedges.append(gp.ET[ep0][1])
        zedges.append(gp.ET[ep0][2])
        # print(ep0)
        # print("x1, y1, z1", gp.ET[ep0])
        ep1 = gp.OC.get_icell(0, gp.OC.alpha(0, e))
        xedges.append(gp.ET[ep1][0])
        yedges.append(gp.ET[ep1][1])
        zedges.append(gp.ET[ep1][2])
        # print(ep1)
        # print("x2, y2, z2", gp.ET[ep1])
        xedges.append(None)
        yedges.append(None)
        zedges.append(None)

    nbtri = 0
    for t in gp.OC.iter_icells(2):
        nbtri += 1
        # print(t)
        cont = True
        t0i = t
        nodes = [nodescorres[gp.OC.get_icell(0, t0i)]]
        while cont:
            t0ip = gp.OC.alpha(1, t0i)
            t0i = gp.OC.alpha(0, t0ip)
            if t == t0i:
                cont = False
            else:
                nodes.append(nodescorres[gp.OC.get_icell(0, t0i)])
        assert len(nodes) == 3
        # print(nodes)
        itri.append(nodes[0])
        jtri.append(nodes[1])
        ktri.append(nodes[2])

    
    d = fig.data[0]
    d.x = xnodes
    d.y = ynodes
    d.z = znodes
    
    e = fig.data[1]
    e.x = xedges
    e.y = yedges
    e.z = zedges
    
    t = fig.data[2]
    t.x = xnodes
    t.y = ynodes
    t.z = znodes
    t.i = itri
    t.j = jtri
    t.k = ktri
    t.intensity = np.random.random(len(itri))

    # d.marker.color = node_adjacencies
    # d.text = node_text
    fig.layout.title = '<br>Subdivised gmap ' + str(nbnodes) + " nodes, " + str(nbedges) + " edges and " + str(nbtri) + " faces"
    
button = widgets.Button(description='update trace')
button.on_click(on_button_click)
# fig.show()

display(button, fig)

Button(description='update trace', style=ButtonStyle())

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': [],
                      …